In [145]:
import pandas as pd
from datetime import datetime, timedelta
import random

# Set the start and end date for data generation (last 5 years from today)
end_date = datetime.now()
start_date = end_date - timedelta(days=5*365)

# Define the departments and their respective surgery volume ranges
departments = {
    'General Surgery Department': (45, 50),
    'Orthopedic Department': (40, 45),
    'Cardiology Department': (30, 35),
    'Gynecology Department': (25, 30),
    'Urology Department': (20, 25),
    'Neurosurgery Department': (15, 20),
    'Ophthalmology Department': (10, 15),
    'ENT Department': (5, 10),
    'Plastic Surgery Department': (3, 8),
    'Oncology Department': (1, 5)
}

# Surgery types for each department
surgery_types = {
    'General Surgery Department': ['Appendectomy', 'Cholecystectomy', 'Hernia Repair', 'Colectomy'],
    'Orthopedic Department': ['Knee Replacement', 'Hip Replacement', 'Spinal Fusion', 'Shoulder Arthroscopy'],
    'Cardiology Department': ['Coronary Artery Bypass Graft', 'Angioplasty', 'Heart Valve Replacement', 'Pacemaker Implantation'],
    'Gynecology Department': ['Hysterectomy', 'Cesarean Section', 'Laparoscopic Surgery for Endometriosis', 'Myomectomy'],
    'Urology Department': ['Prostatectomy', 'Kidney Stone Removal', 'Bladder Surgery', 'Vasectomy'],
    'Neurosurgery Department': ['Craniotomy', 'Spinal Cord Surgery', 'Brain Tumor Removal', 'Deep Brain Stimulation'],
    'Ophthalmology Department': ['Cataract Surgery', 'LASIK Surgery', 'Retinal Detachment Surgery', 'Glaucoma Surgery'],
    'ENT Department': ['Tonsillectomy', 'Sinus Surgery', 'Cochlear Implant Surgery', 'Rhinoplasty'],
    'Plastic Surgery Department': ['Breast Augmentation', 'Rhinoplasty', 'Liposuction', 'Facelift'],
    'Oncology Department': ['Mastectomy', 'Chemotherapy', 'Radiation Therapy', 'Tumor Resection Surgery']
}

distribution_percent = [0.5,0.25,0.15,0.1]

hospitals = ['St. Mary Hospital', 'General Medical Center', 'City Health Center', 'Hope Medical Center']

# Initialize an empty list to store the generated data
data = []

# Generate data for each date within the specified range
current_date = start_date
while current_date <= end_date:

    day_of_week = current_date.weekday()  # 0 = Monday, 6 = Sunday

    for hospital in hospitals:
        # Iterate over each department

        for department, volume_range in departments.items():
            # Get the surgery volume range for the department

            min_volume, max_volume = volume_range

            # Generate a random surgery volume within the range
            surgery_volume = random.randint(min_volume, max_volume) + 5 * day_of_week

            # Get the surgery types for the department
            department_surgery_types = surgery_types[department]

            tem = {i:j for i, j in zip(department_surgery_types,[surgery_volume*i for i in distribution_percent])}

            for surgery,volume in tem.items():

                # Create a row of data with the specified features
                row = [current_date, hospital, department,surgery,volume]

                # Append the row to the data list
                data.append(row)

    # Move to the next date
    current_date += timedelta(days=1)

# Create a DataFrame from the generated data
df = pd.DataFrame(data, columns=['Date', 'Hospital', 'Department', 'Surgery Type','Surgery Volume'])

# Save the dataset to a CSV file
df.to_csv('../data/surgery_volume_dataset.csv', index=False)


In [146]:
import pandas as pd

data = pd.read_csv(r"../data/surgery_volume_dataset.csv")

In [147]:
import pandas as pd

def create_date_features(dataframe, date_column):
    # Convert the date column to datetime type
    dataframe[date_column] = pd.to_datetime(dataframe[date_column],dayfirst=True)
    
    dataframe['day_of_week'] = dataframe[date_column].dt.weekday
    dataframe['Day'] = dataframe[date_column].dt.day
    
    return dataframe

data = create_date_features(data,"Date")


In [148]:
data.columns


Index(['Date', 'Hospital', 'Department', 'Surgery Type', 'Surgery Volume',
       'day_of_week', 'Day'],
      dtype='object')

In [149]:
cat_coumns = ["Hospital","Department","Surgery Type"]

num_column =["day_of_week","Day","Surgery Volume"]

dummies = pd.get_dummies(data[cat_coumns],drop_first=True)

data_new = pd.concat([dummies,data[num_column]],axis=1)

In [150]:
data_new.columns

Index(['Hospital_General Medical Center', 'Hospital_Hope Medical Center',
       'Hospital_St. Mary Hospital', 'Department_ENT Department',
       'Department_General Surgery Department',
       'Department_Gynecology Department',
       'Department_Neurosurgery Department', 'Department_Oncology Department',
       'Department_Ophthalmology Department',
       'Department_Orthopedic Department',
       'Department_Plastic Surgery Department',
       'Department_Urology Department', 'Surgery Type_Appendectomy',
       'Surgery Type_Bladder Surgery', 'Surgery Type_Brain Tumor Removal',
       'Surgery Type_Breast Augmentation', 'Surgery Type_Cataract Surgery',
       'Surgery Type_Cesarean Section', 'Surgery Type_Chemotherapy',
       'Surgery Type_Cholecystectomy', 'Surgery Type_Cochlear Implant Surgery',
       'Surgery Type_Colectomy', 'Surgery Type_Coronary Artery Bypass Graft',
       'Surgery Type_Craniotomy', 'Surgery Type_Deep Brain Stimulation',
       'Surgery Type_Facelift', '

In [151]:
from sklearn.model_selection import train_test_split
from xgboost import XGBRegressor
import pandas as pd
from sklearn.ensemble import RandomForestRegressor

# Splitting data into train test split

target,features = data_new["Surgery Volume"],data_new.drop("Surgery Volume",axis=1)

features = features.astype("int")

x_train, x_test, y_train, y_test = train_test_split(features, target)

model = XGBRegressor()

model.fit(x_train,y_train)

### Evaluating the model

from sklearn.metrics import mean_absolute_error, r2_score,mean_squared_error

predicted = model.predict(x_test)

print("R2 Score is : ",r2_score(predicted,y_test))

print("Mean Squared Error is : ",mean_squared_error(predicted,y_test))

print("Mean Absolute Error is : ",mean_absolute_error(predicted,y_test))

model.save_model(r"..\models\surgeryVolume_model.model")

R2 Score is :  0.9934482845848616
Mean Squared Error is :  0.3885895572742937
Mean Absolute Error is :  0.4169772836484365


In [152]:
model.predict(x_train.iloc[15:16])[0]

2.644399

In [153]:
y_train.iloc[15:16:]


185998    2.55
Name: Surgery Volume, dtype: float64

In [154]:
surgery_volume_sample ={i:[j] for i,j in zip(x_train.columns,[0 for _ in range(len(x_train.columns))])}


In [155]:
def get_surgery_volume_sample():

    surgery_volume_sample = {'Hospital_General Medical Center': [0],
   'Hospital_Hope Medical Center': [0],
   'Hospital_St. Mary Hospital': [0],
   'Department_ENT Department': [0],
   'Department_General Surgery Department': [0],
   'Department_Gynecology Department': [0],
   'Department_Neurosurgery Department': [0],
   'Department_Oncology Department': [0],
   'Department_Ophthalmology Department': [0],
   'Department_Orthopedic Department': [0],
   'Department_Plastic Surgery Department': [0],
   'Department_Urology Department': [0],
   'Surgery Type_Appendectomy': [0],
   'Surgery Type_Bladder Surgery': [0],
   'Surgery Type_Brain Tumor Removal': [0],
   'Surgery Type_Breast Augmentation': [0],
   'Surgery Type_Cataract Surgery': [0],
   'Surgery Type_Cesarean Section': [0],
   'Surgery Type_Chemotherapy': [0],
   'Surgery Type_Cholecystectomy': [0],
   'Surgery Type_Cochlear Implant Surgery': [0],
   'Surgery Type_Colectomy': [0],
   'Surgery Type_Coronary Artery Bypass Graft': [0],
   'Surgery Type_Craniotomy': [0],
   'Surgery Type_Deep Brain Stimulation': [0],
   'Surgery Type_Facelift': [0],
   'Surgery Type_Glaucoma Surgery': [0],
   'Surgery Type_Heart Valve Replacement': [0],
   'Surgery Type_Hernia Repair': [0],
   'Surgery Type_Hip Replacement': [0],
   'Surgery Type_Hysterectomy': [0],
   'Surgery Type_Kidney Stone Removal': [0],
   'Surgery Type_Knee Replacement': [0],
   'Surgery Type_LASIK Surgery': [0],
   'Surgery Type_Laparoscopic Surgery for Endometriosis': [0],
   'Surgery Type_Liposuction': [0],
   'Surgery Type_Mastectomy': [0],
   'Surgery Type_Myomectomy': [0],
   'Surgery Type_Pacemaker Implantation': [0],
   'Surgery Type_Prostatectomy': [0],
   'Surgery Type_Radiation Therapy': [0],
   'Surgery Type_Retinal Detachment Surgery': [0],
   'Surgery Type_Rhinoplasty': [0],
   'Surgery Type_Shoulder Arthroscopy': [0],
   'Surgery Type_Sinus Surgery': [0],
   'Surgery Type_Spinal Cord Surgery': [0],
   'Surgery Type_Spinal Fusion': [0],
   'Surgery Type_Tonsillectomy': [0],
   'Surgery Type_Tumor Resection Surgery': [0],
   'Surgery Type_Vasectomy': [0],
   'day_of_week': [0],
   'Day': [0]}

    return surgery_volume_sample

In [156]:
for dept in departments.keys():


    t = "Department_"+dept

    if t not in surgery_volume_sample:

        print(t)

Department_Cardiology Department


In [157]:
all_surgery = list(surgery_types.values())

all_surgery = [i for j in all_surgery for i in j]

for dept in all_surgery:

    t = "Surgery Type_"+dept

    if t not in surgery_volume_sample:

        print(t)

Surgery Type_Angioplasty


In [158]:
for dept in departments.keys():


    t = "Department_"+dept

    if t not in surgery_volume_sample:

        print(t)

Department_Cardiology Department


In [159]:
def prepare_surgery_sample(current_date,department,surgery_type):
    
    surgery_volume_sample = get_surgery_volume_sample()
    
    surgery_volume_sample['day_of_week'] = current_date.weekday() 
    surgery_volume_sample['Day'] = current_date.day
    if department =="Department_Cardiology Department":

        print("Error 1")
        
        pass
        
    else:
            
        surgery_volume_sample[department] = [1]

    if surgery_type =="Surgery Type_Angioplasty":

        print("Error 2")

        pass
    else:
        surgery_volume_sample[surgery_type] = [1]


    return pd.DataFrame(surgery_volume_sample)


In [160]:
prepare_surgery_sample(datetime.today(),"Department_Oncology Department","Surgery Type_Sinus Surgery")

,Hospital_General Medical Center,Hospital_Hope Medical Center,Hospital_St. Mary Hospital,Department_ENT Department,Department_General Surgery Department,Department_Gynecology Department,Department_Neurosurgery Department,Department_Oncology Department,Department_Ophthalmology Department,Department_Orthopedic Department,...,Surgery Type_Rhinoplasty,Surgery Type_Shoulder Arthroscopy,Surgery Type_Sinus Surgery,Surgery Type_Spinal Cord Surgery,Surgery Type_Spinal Fusion,Surgery Type_Tonsillectomy,Surgery Type_Tumor Resection Surgery,Surgery Type_Vasectomy,day_of_week,Day
0,0,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,1,11


In [161]:
prepare_surgery_sample(datetime.today()+timedelta(days=1),"Department_Oncology Department","Surgery Type_Sinus Surgery")


,Hospital_General Medical Center,Hospital_Hope Medical Center,Hospital_St. Mary Hospital,Department_ENT Department,Department_General Surgery Department,Department_Gynecology Department,Department_Neurosurgery Department,Department_Oncology Department,Department_Ophthalmology Department,Department_Orthopedic Department,...,Surgery Type_Rhinoplasty,Surgery Type_Shoulder Arthroscopy,Surgery Type_Sinus Surgery,Surgery Type_Spinal Cord Surgery,Surgery Type_Spinal Fusion,Surgery Type_Tonsillectomy,Surgery Type_Tumor Resection Surgery,Surgery Type_Vasectomy,day_of_week,Day
0,0,0,0,0,0,0,0,1,0,0,...,0,0,1,0,0,0,0,0,2,12


In [180]:
def get_next_fewdays_surgery_demand():

    surgery_output ={

        "Next day":{
        'General Surgery Department': {
            'Appendectomy': 0,
            'Cholecystectomy': 0,
            'Hernia Repair': 0,
            'Colectomy': 0
        },
        'Orthopedic Department': {
            'Knee Replacement': 0,
            'Hip Replacement': 0,
            'Spinal Fusion': 0,
            'Shoulder Arthroscopy': 0
        },
        'Cardiology Department': {
            'Coronary Artery Bypass Graft': 0,
            'Angioplasty': 0,
            'Heart Valve Replacement': 0,
            'Pacemaker Implantation': 0
        },
        'Gynecology Department': {
            'Hysterectomy': 0,
            'Cesarean Section': 0,
            'Laparoscopic Surgery for Endometriosis': 0,
            'Myomectomy': 0
        },
        'Urology Department': {
            'Prostatectomy': 0,
            'Kidney Stone Removal': 0,
            'Bladder Surgery': 0,
            'Vasectomy': 0
        },
        'Neurosurgery Department': {
            'Craniotomy': 0,
            'Spinal Cord Surgery': 0,
            'Brain Tumor Removal': 0,
            'Deep Brain Stimulation': 0
        },
        'Ophthalmology Department': {
            'Cataract Surgery': 0,
            'LASIK Surgery': 0,
            'Retinal Detachment Surgery': 0,
            'Glaucoma Surgery': 0
        },
        'ENT Department': {
            'Tonsillectomy': 0,
            'Sinus Surgery': 0,
            'Cochlear Implant Surgery': 0,
            'Rhinoplasty': 0
        },
        'Plastic Surgery Department': {
            'Breast Augmentation': 0,
            'Rhinoplasty': 0,
            'Liposuction': 0,
            'Facelift': 0
        },
        'Oncology Department': {
            'Mastectomy': 0,
            'Chemotherapy': 0,
            'Radiation Therapy': 0,
            'Tumor Resection Surgery': 0
        }
    },
        "Next 2 days":{
        'General Surgery Department': {
            'Appendectomy': 0,
            'Cholecystectomy': 0,
            'Hernia Repair': 0,
            'Colectomy': 0
        },
        'Orthopedic Department': {
            'Knee Replacement': 0,
            'Hip Replacement': 0,
            'Spinal Fusion': 0,
            'Shoulder Arthroscopy': 0
        },
        'Cardiology Department': {
            'Coronary Artery Bypass Graft': 0,
            'Angioplasty': 0,
            'Heart Valve Replacement': 0,
            'Pacemaker Implantation': 0
        },
        'Gynecology Department': {
            'Hysterectomy': 0,
            'Cesarean Section': 0,
            'Laparoscopic Surgery for Endometriosis': 0,
            'Myomectomy': 0
        },
        'Urology Department': {
            'Prostatectomy': 0,
            'Kidney Stone Removal': 0,
            'Bladder Surgery': 0,
            'Vasectomy': 0
        },
        'Neurosurgery Department': {
            'Craniotomy': 0,
            'Spinal Cord Surgery': 0,
            'Brain Tumor Removal': 0,
            'Deep Brain Stimulation': 0
        },
        'Ophthalmology Department': {
            'Cataract Surgery': 0,
            'LASIK Surgery': 0,
            'Retinal Detachment Surgery': 0,
            'Glaucoma Surgery': 0
        },
        'ENT Department': {
            'Tonsillectomy': 0,
            'Sinus Surgery': 0,
            'Cochlear Implant Surgery': 0,
            'Rhinoplasty': 0
        },
        'Plastic Surgery Department': {
            'Breast Augmentation': 0,
            'Rhinoplasty': 0,
            'Liposuction': 0,
            'Facelift': 0
        },
        'Oncology Department': {
            'Mastectomy': 0,
            'Chemotherapy': 0,
            'Radiation Therapy': 0,
            'Tumor Resection Surgery': 0
        }
    },
        "Next 3 days":{
        'General Surgery Department': {
            'Appendectomy': 0,
            'Cholecystectomy': 0,
            'Hernia Repair': 0,
            'Colectomy': 0
        },
        'Orthopedic Department': {
            'Knee Replacement': 0,
            'Hip Replacement': 0,
            'Spinal Fusion': 0,
            'Shoulder Arthroscopy': 0
        },
        'Cardiology Department': {
            'Coronary Artery Bypass Graft': 0,
            'Angioplasty': 0,
            'Heart Valve Replacement': 0,
            'Pacemaker Implantation': 0
        },
        'Gynecology Department': {
            'Hysterectomy': 0,
            'Cesarean Section': 0,
            'Laparoscopic Surgery for Endometriosis': 0,
            'Myomectomy': 0
        },
        'Urology Department': {
            'Prostatectomy': 0,
            'Kidney Stone Removal': 0,
            'Bladder Surgery': 0,
            'Vasectomy': 0
        },
        'Neurosurgery Department': {
            'Craniotomy': 0,
            'Spinal Cord Surgery': 0,
            'Brain Tumor Removal': 0,
            'Deep Brain Stimulation': 0
        },
        'Ophthalmology Department': {
            'Cataract Surgery': 0,
            'LASIK Surgery': 0,
            'Retinal Detachment Surgery': 0,
            'Glaucoma Surgery': 0
        },
        'ENT Department': {
            'Tonsillectomy': 0,
            'Sinus Surgery': 0,
            'Cochlear Implant Surgery': 0,
            'Rhinoplasty': 0
        },
        'Plastic Surgery Department': {
            'Breast Augmentation': 0,
            'Rhinoplasty': 0,
            'Liposuction': 0,
            'Facelift': 0
        },
        'Oncology Department': {
            'Mastectomy': 0,
            'Chemotherapy': 0,
            'Radiation Therapy': 0,
            'Tumor Resection Surgery': 0
        }
    }
    
    }


    from datetime import datetime

    from datetime import timedelta

    today = datetime.today()
    
    tomorrow = today+timedelta(days=1)

    day_after_tomorrow = today+timedelta(days=2)

    next_to_DFT = today+timedelta(days=3)

    next_few_days = [tomorrow,day_after_tomorrow,next_to_DFT]

    for ind,date in enumerate(next_few_days):

        for department in surgery_output["Next day"].keys():

            for surgery_type in surgery_output["Next day"][department]:

                dep = "Department_"+department

                ST = "Surgery Type_"+surgery_type

                input = prepare_surgery_sample(date,dep,ST)

                surgery_count = int(model.predict(input)[0])

                if department in surgery_output[list(surgery_output.keys())[ind]].keys():

                    surgery_output[list(surgery_output.keys())[ind]][department][surgery_type] = surgery_count

                else:

                    surgery_output[list(surgery_output.keys())[ind]]['Department_Cardiology Department'][surgery_type] = surgery_count

                    
    return surgery_output


In [181]:
out = get_next_fewdays_surgery_demand()

Error 1
Error 1
Error 2
Error 1
Error 1
Error 1
Error 1
Error 2
Error 1
Error 1
Error 1
Error 1
Error 2
Error 1
Error 1


In [182]:
def get_total_output(admission_output):
    
    output = {}
    
    for day in admission_output.keys():
        
        total = 0
        
        for department in admission_output[day].keys():

            for surgery_type in admission_output[day][department].keys():
            
                total+=admission_output[day][department][surgery_type]
            
        output[day] = total
        
    return output

In [183]:
get_total_output(out)

{'Next day': 298, 'Next 2 days': 348, 'Next 3 days': 396}

In [184]:
out

{'Next day': {'General Surgery Department': {'Appendectomy': 28,
   'Cholecystectomy': 14,
   'Hernia Repair': 8,
   'Colectomy': 5},
  'Orthopedic Department': {'Knee Replacement': 26,
   'Hip Replacement': 13,
   'Spinal Fusion': 7,
   'Shoulder Arthroscopy': 5},
  'Cardiology Department': {'Coronary Artery Bypass Graft': 21,
   'Angioplasty': 8,
   'Heart Valve Replacement': 6,
   'Pacemaker Implantation': 4},
  'Gynecology Department': {'Hysterectomy': 18,
   'Cesarean Section': 9,
   'Laparoscopic Surgery for Endometriosis': 5,
   'Myomectomy': 3},
  'Urology Department': {'Prostatectomy': 16,
   'Kidney Stone Removal': 8,
   'Bladder Surgery': 4,
   'Vasectomy': 3},
  'Neurosurgery Department': {'Craniotomy': 13,
   'Spinal Cord Surgery': 6,
   'Brain Tumor Removal': 4,
   'Deep Brain Stimulation': 2},
  'Ophthalmology Department': {'Cataract Surgery': 11,
   'LASIK Surgery': 5,
   'Retinal Detachment Surgery': 3,
   'Glaucoma Surgery': 2},
  'ENT Department': {'Tonsillectomy': 8

In [2]:
numbers = [2,6,8,7,1]

target = 8

def get(numbers,target):

    for ind, element in enumerate(numbers):

        for ind1, element1 in enumerate(numbers):

            if element+element1 == target:

                return [ind,ind1]
            
get(numbers,target)

[0, 1]